# Pipeline for analyzing beepollen data (Regional database)

Author: Xiaoping Li  
Organization: Oregon State University Hermiston Agricultural Research and Extension Center

##### Note: before you start analyzing, move 1. metaBarTools.py 2. meta file (.csv) 3. plate file (.xlsx) to your working directory.

In [1]:
import os
from metaBarTools import metaBar_PreX

In [2]:
metaBar = metaBar_PreX()

In [3]:
# reads path
ITS_reads = os.path.abspath("./Regional_reads/ITS-reads")
rbcL_reads = os.path.abspath("./Regional_reads/rbcL-reads/")

In [4]:
# setup path files

platesetup = os.path.abspath('./beepollen_all.xlsx')
metafile = os.path.abspath('./meta_beepollen_all.csv')

In [5]:
# set up subdirectories for analysis
path_ITS, path_rbcL_pair, path_rbcL_single = metaBar.metaBar_makeSubDir("Regional_Results", ["ITS_result", "rbcL_result", "rbcL_result_single"])


        ####################################################
        #########          metaBar_makeSubDir     ##########
        ####################################################
        


In [6]:
ITS_ref_seq = os.path.abspath("./database_beepollen/Regional/ITS2_Regional.fasta")
ITS_ref_mapping = os.path.abspath("./database_beepollen/Regional/ITS2_Regional.mapping")
rbcL_ref_seq = os.path.abspath("./database_beepollen/Regional/rbcL_Regional.fasta")
rbcL_ref_mapping = os.path.abspath("./database_beepollen/Regional/rbcL_Regional.mapping")

## With regional ITS2 database

In [7]:
# change directory into Analysis_results/ITS_results
os.chdir(path_ITS)

In [ ]:
# get primers lengths for trimming in DADA2

ITS_f_len = len("ATGCGATACTTGGTGTGAAT")
ITS_r_len = len("TCCTCCGCTTATTGATATGC")

In [ ]:
# create manifest file for qiime2 to find the reads
manifest = metaBar.metaBar_Qiime2_Manifest(ITS_reads, platesetup, sheetname=0, matchby="sample")

In [ ]:
# change the name for the manifest file, so it is more readable 
!mv ITSS2F@ITS4R_manifest.csv beepollen_ITS2_manifest.csv

In [ ]:
# import data

!qiime tools import \
--type 'SampleData[PairedEndSequencesWithQuality]' \
--input-path ./beepollen_ITS2_manifest.csv \
--output-path beepollen.qza \
--input-format PairedEndFastqManifestPhred33

In [ ]:
# make folders to store dada2 and feature-table results

if not os.path.exists("dada2-stats"):
    os.makedirs("dada2-stats")
    
if not os.path.exists("feature-tables"):
    os.makedirs("feature-tables")

In [ ]:
# run dada2 for denoising, set truncating quality cutoff at 22

!!qiime dada2 denoise-paired \
--i-demultiplexed-seqs beepollen.qza \
--output-dir dada2 \
--o-table feature-tables/table-beepollen \
--o-representative-sequences rep_seq_beepollen \
--p-trim-left-f $ITS_f_len \
--p-trim-left-r $ITS_r_len \
--p-trunc-len-f 299 \
--p-trunc-len-r 243 \
--p-n-threads 12 \
--o-denoising-stats dada2-stats/dada2_stats.qza

In [ ]:
# visualize the dada2 stats

!qiime metadata tabulate \
--m-input-file dada2-stats/dada2_stats.qza \
--o-visualization dada2-stats/dada2_stats.qzv

In [ ]:
# visualize the feature table (OTU table)

!qiime feature-table summarize \
--i-table feature-tables/table-beepollen.qza \
--o-visualization feature-tables/table-beepollen.qzv \
--m-sample-metadata-file $metafile

In [ ]:
# filter low feature counts for better visualization

!qiime feature-table filter-features \
--i-table feature-tables/table-beepollen.qza \
--p-min-frequency 1000 \
--p-min-samples 4 \
--o-filtered-table feature-tables/filtered_table.qza

In [ ]:
# use sklearn NB classifier
if not os.path.exists("classifier"):
    os.makedirs("classifier")

In [ ]:
# import reference sequence
!qiime tools import \
--type 'FeatureData[Sequence]' \
--input-path $ITS_ref_seq \
--output-path ./classifier/ITS2_starky_db_rep.qza

In [ ]:
# import reference sequence taxonomy mapping
!qiime tools import \
--type 'FeatureData[Taxonomy]' \
--input-format HeaderlessTSVTaxonomyFormat \
--input-path $ITS_ref_mapping \
--output-path ./classifier/ITS2_starky_db_taxonomy.qza

In [ ]:
if not os.path.exists("taxonomy"):
    os.makedirs("taxonomy")

In [ ]:
# train classifier
!qiime feature-classifier fit-classifier-naive-bayes \
--i-reference-reads classifier/ITS2_starky_db_rep.qza \
--i-reference-taxonomy classifier/ITS2_starky_db_taxonomy.qza \
--o-classifier classifier/classifier_ITS2_starkey_db_default.qza

In [ ]:
# assign taxonomy
!qiime feature-classifier classify-sklearn \
--i-classifier classifier/classifier_ITS2_starkey_db_default.qza \
--i-reads rep_seq_beepollen.qza \
--o-classification ./taxonomy/taxonomy_beepollen_ITS2_default.qza

In [ ]:
# making relative taxonomy barplot
!qiime taxa barplot \
--i-table feature-tables/filtered_table.qza \
--i-taxonomy taxonomy/taxonomy_beepollen_ITS2_default.qza \
--m-metadata-file $metafile \
--o-visualization taxonomy/barplot_beepollen_ITS2_default.qzv

## With regional *rbcL* database 

### Paired End

In [8]:
# change to rbcL analysis folder
os.chdir(path_rbcL_pair)

In [9]:
# rbcL primers' lengths

rbcL_f_len = len("TGGCAGCATTYCGAGTAACTC")
rbcL_r_len = len("GTAAAATCAAGTCCACCRCG")

In [10]:
# making manifest file for rbcL
manifest_rbcL = metaBar.metaBar_Qiime2_Manifest(rbcL_reads, platesetup, sheetname=0, matchby="sample")


        ####################################################
        #########     metaBar_Qiime2_Manifest     ##########
        ####################################################
        
# of reads: 1114
# of reads in the manifest file: 1114

Manifest Completed


In [11]:
!mv ITSS2F@ITS4R_manifest.csv beepollen_rbcL_manifest.csv

In [12]:
# import reads into pipeline
!qiime tools import \
--type 'SampleData[PairedEndSequencesWithQuality]' \
--input-path ./beepollen_rbcL_manifest.csv \
--output-path beepollen.qza \
--input-format PairedEndFastqManifestPhred33

Imported ./beepollen_rbcL_manifest.csv as PairedEndFastqManifestPhred33 to beepollen.qza


In [13]:
# check sequence quality
!qiime demux summarize \
--i-data beepollen.qza \
--o-visualization beepollen_seqs_rbcL.qzv

Saved Visualization to: beepollen_seqs_rbcL.qzv


In [14]:
# make folders to store dada2 and feature-table results

if not os.path.exists("dada2-stats"):
    os.makedirs("dada2-stats")
    
if not os.path.exists("feature-tables"):
    os.makedirs("feature-tables")

In [15]:
# dada2 denosing
# truncate quality cutoff 22

!qiime dada2 denoise-paired \
--i-demultiplexed-seqs beepollen.qza \
--output-dir dada2 \
--o-table feature-tables/table-beepollen \
--o-representative-sequences rep_seq_beepollen \
--p-trim-left-f $rbcL_f_len \
--p-trim-left-r $rbcL_r_len \
--p-trunc-len-f 299 \
--p-trunc-len-r 268 \
--p-n-threads 12 \
--o-denoising-stats dada2-stats/dada2_stats.qza


Saved FeatureTable[Frequency] to: feature-tables/table-beepollen.qza
Saved FeatureData[Sequence] to: rep_seq_beepollen.qza
Saved SampleData[DADA2Stats] to: dada2-stats/dada2_stats.qza


In [16]:
# output dada2 stats visualization

!qiime metadata tabulate \
--m-input-file dada2-stats/dada2_stats.qza \
--o-visualization dada2-stats/dada2_stats.qzv

Saved Visualization to: dada2-stats/dada2_stats.qzv


In [17]:
# filter OTU table: minimun feature for the sample: 1000, and the feature has to be in at least 4 samples

!qiime feature-table filter-features \
--i-table feature-tables/table-beepollen.qza \
--p-min-frequency 1000 \
--p-min-samples 4 \
--o-filtered-table feature-tables/filtered_table.qza

Saved FeatureTable[Frequency] to: feature-tables/filtered_table.qza


In [18]:
if not os.path.exists("classifier"):
    os.makedirs("classifier")

In [19]:
# import reference sequences

!qiime tools import \
--type 'FeatureData[Sequence]' \
--input-path $rbcL_ref_seq \
--output-path ./classifier/rbcL_refined_db.qza

Imported /media/swaggyp1985/HDD4T/OSU_Projects_2017-2018/running_project/beepollen_github/database_beepollen/Regional/rbcL_Regional.fasta as DNASequencesDirectoryFormat to ./classifier/rbcL_refined_db.qza


In [21]:
# import reference taxonomy mapping

!qiime tools import \
--type 'FeatureData[Taxonomy]' \
--input-format HeaderlessTSVTaxonomyFormat \
--input-path $rbcL_ref_mapping \
--output-path ./classifier/rbcL_refined_taxonomy.qza

Imported /media/swaggyp1985/HDD4T/OSU_Projects_2017-2018/running_project/beepollen_github/database_beepollen/Regional/rbcL_Regional.mapping as HeaderlessTSVTaxonomyFormat to ./classifier/rbcL_refined_taxonomy.qza


In [22]:
if not os.path.exists("taxonomy"):
    os.makedirs("taxonomy")

In [23]:
# train classifier: naive bayes

!qiime feature-classifier fit-classifier-naive-bayes \
--i-reference-reads ./classifier/rbcL_refined_db.qza \
--i-reference-taxonomy ./classifier/rbcL_refined_taxonomy.qza \
--o-classifier ./classifier/classifier_rbcL_refined_default.qza

Saved TaxonomicClassifier to: ./classifier/classifier_rbcL_refined_default.qza


In [24]:
# assign taxonomy

!qiime feature-classifier classify-sklearn \
--i-classifier ./classifier/classifier_rbcL_refined_default.qza \
--i-reads rep_seq_beepollen.qza \
--o-classification ./taxonomy/taxonomy_rbcL_default

Saved FeatureData[Taxonomy] to: ./taxonomy/taxonomy_rbcL_default.qza


In [25]:
# visualize taxonomy table

!qiime metadata tabulate \
--m-input-file taxonomy/taxonomy_rbcL_default.qza \
--o-visualization taxonomy/taxonomy_rbcL_default.qzv

Saved Visualization to: taxonomy/taxonomy_rbcL_default.qzv


In [26]:
# taxonomy relative frequency in barplot

!qiime taxa barplot \
--i-table feature-tables/filtered_table.qza \
--i-taxonomy taxonomy/taxonomy_rbcL_default.qza \
--m-metadata-file ../../meta_beepollen_all.csv \
--o-visualization taxonomy/barplot_rbcL_default.qzv

Saved Visualization to: taxonomy/barplot_rbcL_default.qzv


### single end

In [27]:
os.chdir(path_rbcL_single)

In [28]:
# making manifest file
manifest_rbcL_single = metaBar.metaBar_Qiime2_Manifest(rbcL_reads, platesetup, sheetname=0, matchby="sample", paired=False)


        ####################################################
        #########     metaBar_Qiime2_Manifest     ##########
        ####################################################
        
# of reads: 557
# of reads in the manifest file: 557

Manifest Completed


In [29]:
!mv ITSS2F@ITS4R_manifest.csv rbcl_single_manifest.csv

In [30]:
rbcL_f_len = len("TGGCAGCATTYCGAGTAACTC")

In [35]:
if not os.path.exists("dada2-stats"):
    os.makedirs("dada2-stats")
    
if not os.path.exists("feature-tables"):
    os.makedirs("feature-tables")

In [31]:
# import forward sequences into qiime2

!qiime tools import \
--type 'SampleData[SequencesWithQuality]' \
--input-path ./rbcl_single_manifest.csv \
--output-path single_rbcl_beepollen.qza \
--input-format SingleEndFastqManifestPhred33

Imported ./rbcl_single_manifest.csv as SingleEndFastqManifestPhred33 to single_rbcl_beepollen.qza


In [33]:
# view summarise of the quality
!qiime demux summarize \
--i-data single_rbcl_beepollen.qza \
--o-visualization single_beepollen_seqs_rbcL.qzv

['Saved Visualization to: single_beepollen_seqs_rbcL.qzv']

In [36]:
# dada2 for single end

!qiime dada2 denoise-single \
--i-demultiplexed-seqs single_rbcl_beepollen.qza \
--output-dir dada2 \
--o-table table-se-beepollen \
--o-representative-sequences rep_se_seq_beepollen \
--p-trim-left $rbcL_f_len \
--p-trunc-len 299 \
--p-n-threads 12 \
--o-denoising-stats dada2-stats/dada2_stats.qza

Saved FeatureTable[Frequency] to: table-se-beepollen.qza
Saved FeatureData[Sequence] to: rep_se_seq_beepollen.qza
Saved SampleData[DADA2Stats] to: dada2-stats/dada2_stats.qza


In [37]:
# filter table
!qiime feature-table filter-features \
--i-table table-se-beepollen.qza \
--p-min-frequency 1000 \
--p-min-samples 4 \
--o-filtered-table filtered_table.qza

Saved FeatureTable[Frequency] to: filtered_table.qza


In [38]:
if not os.path.exists("classifier"):
    os.makedirs("classifier")
    
if not os.path.exists("taxonomy"):
    os.makedirs("taxonomy")

In [39]:
# import referece sequences into qiime2

!qiime tools import \
--type 'FeatureData[Sequence]' \
--input-path $rbcL_ref_seq \
--output-path ./classifier/rbcL_refined_db.qza

Imported /media/swaggyp1985/HDD4T/OSU_Projects_2017-2018/running_project/beepollen_github/database_beepollen/Regional/rbcL_Regional.fasta as DNASequencesDirectoryFormat to ./classifier/rbcL_refined_db.qza


In [40]:
# import reference taxonomy mapping into qiime2

!qiime tools import \
--type 'FeatureData[Taxonomy]' \
--input-format HeaderlessTSVTaxonomyFormat \
--input-path $rbcL_ref_mapping \
--output-path ./classifier/rbcL_refined_taxonomy.qza

Imported /media/swaggyp1985/HDD4T/OSU_Projects_2017-2018/running_project/beepollen_github/database_beepollen/Regional/rbcL_Regional.mapping as HeaderlessTSVTaxonomyFormat to ./classifier/rbcL_refined_taxonomy.qza


In [41]:
# train classifier

!qiime feature-classifier fit-classifier-naive-bayes \
--i-reference-reads ./classifier/rbcL_refined_db.qza \
--i-reference-taxonomy ./classifier/rbcL_refined_taxonomy.qza \
--o-classifier ./classifier/classifier_rbcL_refined_default.qza

Saved TaxonomicClassifier to: ./classifier/classifier_rbcL_refined_default.qza


In [42]:
# assign taxonomy

!qiime feature-classifier classify-sklearn \
--i-classifier ./classifier/classifier_rbcL_refined_default.qza \
--i-reads rep_se_seq_beepollen.qza \
--o-classification ./taxonomy/taxonomy_rbcL_default.qza

Saved FeatureData[Taxonomy] to: ./taxonomy/taxonomy_rbcL_default.qza


In [43]:
# visualizing taxonomy assignment

!qiime metadata tabulate \
--m-input-file taxonomy/taxonomy_rbcL_default.qza \
--o-visualization taxonomy/taxonomy_rbcL_default.qzv

Saved Visualization to: taxonomy/taxonomy_rbcL_default.qzv


In [46]:
# taxonomy composition

!qiime taxa barplot \
--i-table filtered_table.qza \
--i-taxonomy taxonomy/taxonomy_rbcL_default.qza \
--m-metadata-file $metafile \
--o-visualization taxonomy/barplot_rbcL_default.qzv

Saved Visualization to: taxonomy/barplot_rbcL_default.qzv
